Import libraries

Import libraries

In [2]:
import requests
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio 
pio.renderers.default = "iframe_connected"
from plotly.subplots import make_subplots

import json
import os

import logging
import scrapy
from scrapy.crawler import CrawlerProcess

import numpy as np
from urllib.parse import urlencode
from urllib.parse import urlparse
import re

from bs4 import BeautifulSoup

Cities list

In [3]:
cities_france = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg","Chateau du Haut Koenigsbourg",
"Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon","Bormes les Mimosas","Cassis","Marseille","Aix en Provence",
"Avignon","Uzes","Nimes","Aigues Mortes","Saintes Maries de la mer","Collioure","Carcassonne","Ariege","Toulouse","Montauban","Biarritz",
"Bayonne","La Rochelle"]

Create class to get location of cities

In [4]:
class geolocalisation_cities():
    
    def __init__(self, cities_list):
        self.cities_list = cities_list
       
    
    def cities_france(self):
        
        dico ={}
        lat =[]
        lon =[]
        cities =[]
        id_cities = []
        i=1
        
        for city in self.cities_list : 
            r=requests.get('https://nominatim.openstreetmap.org/search?q='+city+"&format=json&limit=1")
            geo = r.json()
        
            lat.append(geo[0]["lat"])
            lon.append(geo[0]["lon"])
            cities.append(city)
            id_cities.append(i)
            i+=1
        
        dico["id_city"] = id_cities
        dico["city"] = cities
        dico["latitude"] = lat
        dico["longitude"] = lon
    
        return dico

    
    def dataframe_cities(self):
        data = pd.DataFrame(self.cities_france())
        return data

Create class to get 7-day weather data

In [9]:
class meteo_cities():
    
    def __init__(self, meteo_cities):
        self.meteo_cities = meteo_cities
    
    
    def meteo_dico(self):
        towns= {}

        for index, row  in self.meteo_cities.iterrows():
            
            self.meteo_cities.loc[index,"city"] +" "+ self.meteo_cities.loc[index,"latitude"]
            city =  self.meteo_cities.loc[index,"city"]
            lat1 =  self.meteo_cities.loc[index,"latitude"]
            lon1 = self.meteo_cities.loc[index,"longitude"]

            rm= requests.get("https://api.openweathermap.org/data/2.5/onecall?lat="+lat1+"&lon="+lon1+"&units=metric&exclude=minutely,hourly&appid=9ad86c2dca5c9025784ab49525637a54")
            data_json = rm.json()
            towns[city] = data_json["daily"]
            
        return towns
        
    
    
    def meteo_dataframe(self):
        meteo_final ={}
        cities =[]
        mean_cloud =[]
        mean_prec = []
        mean_temp_day =[]
        id_city =[]
        index = 1



        
        prec =[]
        clouds =[]
        temp_day =[]
        
        dico_meteo ={}
        

        for k, values in self.meteo_dico().items():
            for i, v in enumerate(self.meteo_dico()[k]):
                
                
                prec.append(v["pop"])
                clouds.append(v["clouds"])
                temp_day.append(v["temp"]["day"])
    
           
            dico_meteo["Probability of precipitation"] = prec
            dico_meteo["clouds"] = clouds
            dico_meteo["day temperature"] = temp_day

            data_meteo= pd.DataFrame(dico_meteo)
            mean_data = data_meteo.mean()
            cities.append(k)
            mean_cloud.append(mean_data.loc["clouds"])
            mean_prec.append(mean_data.loc["Probability of precipitation"])
            mean_temp_day.append(mean_data.loc["day temperature"])
            id_city.append(index)
            index+=1
    
    
    
        meteo_final["id_city"]=id_city 
        meteo_final["city"] = cities
        meteo_final["Average probability of precipitation"] = mean_prec
        meteo_final["average cloud percentage"] = mean_cloud
        meteo_final["average daily temperatures"] = mean_temp_day
        meteo_final_df = pd.DataFrame(meteo_final)
        
        return meteo_final_df

In [6]:
cities_1 = geolocalisation_cities(cities_france)


Get latitude and longitude for each cities

In [7]:
cities_lat_lon= cities_1.dataframe_cities()
cities_lat_lon

,id_city,city,latitude,longitude
0,1,Mont Saint Michel,48.6359541,-1.511459954959514
1,2,St Malo,48.649518,-2.0260409
2,3,Bayeux,49.2764624,-0.7024738
3,4,Le Havre,49.4938975,0.1079732
4,5,Rouen,49.4404591,1.0939658
5,6,Paris,48.8588897,2.3200410217200766
6,7,Amiens,49.8941708,2.2956951
7,8,Lille,50.6365654,3.0635282
8,9,Strasbourg,48.584614,7.7507127
9,10,Chateau du Haut Koenigsbourg,48.249489800000006,7.34429620253195


Get weather data for each cities

In [ ]:
meteo = meteo_cities(cities_lat_lon)
weather_cities = meteo.meteo_dataframe()

weather_cities

Merge the two dataframes

In [ ]:
data_weather_cities =cities_lat_lon.merge(weather_cities, on = ['id_city',"city"])
data_weather_cities[['latitude', 'longitude']]=data_weather_cities[['latitude', 'longitude']].astype(float)
data_weather_cities

,id_city,city,latitude,longitude,Average probability of precipitation,average cloud percentage,average daily temperatures
0,1,Mont Saint Michel,48.635954,-1.511460,0.571250,74.375000,15.141250
1,2,St Malo,48.649518,-2.026041,0.578125,75.437500,15.140000
2,3,Bayeux,49.276462,-0.702474,0.610000,75.250000,15.012083
3,4,Le Havre,49.493898,0.107973,0.651875,78.343750,14.883125
4,5,Rouen,49.440459,1.093966,0.658250,78.200000,14.647500
5,6,Paris,48.858890,2.320041,0.674583,76.416667,14.561667
6,7,Amiens,49.894171,2.295695,0.687143,76.500000,14.316786
7,8,Lille,50.636565,3.063528,0.697500,76.593750,14.120625
8,9,Strasbourg,48.584614,7.750713,0.665972,76.833333,14.064167
9,10,Chateau du Haut Koenigsbourg,48.249490,7.344296,0.644750,76.475000,13.736375


Ranking of the cities according to the weather

In [ ]:
data_weather_cities_class = data_weather_cities.sort_values(by=['Average probability of precipitation','average cloud percentage',"average daily temperatures"],ascending=[True,True,False])

data_weather_cities_class

Top 5 cities in the ranking

In [ ]:
top_5 = data_weather_cities_class.iloc[0:5]
top_5

,id_city,city,latitude,longitude,Average probability of precipitation,average cloud percentage,average daily temperatures
31,32,Montauban,44.017584,1.354999,0.428164,61.769531,15.677773
32,33,Biarritz,43.471144,-1.552727,0.429242,61.443182,15.741098
30,31,Toulouse,43.604462,1.444247,0.429677,61.991935,15.605242
33,34,Bayonne,43.494514,-1.473666,0.430257,61.099265,15.818640
29,30,Ariege,42.945537,1.406554,0.435417,62.195833,15.524125


In [ ]:
fig = px.scatter_mapbox(top_5, lat="latitude", lon="longitude", color="city",
                        mapbox_style="open-street-map",                    
                        zoom=10,
                        height=600)
fig.update_layout(
    
    margin=dict(l=30, r=30, t=30, b=30),
    paper_bgcolor="LightSteelBlue",
)
fig.show()
fig.write_html("top5.html")

Scrapping with scrapy

In [ ]:
filename = "scrap_booking.json" 

if filename in os.listdir():
        os.remove(filename)

user_agent = 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:98.0) Gecko/20100101 Firefox/98.0'
process = CrawlerProcess(settings = {
    'USER_AGENT': user_agent,
    'LOG_LEVEL': logging.ERROR,
    "FEEDS": { filename : {"format": "json"}
             }
})

In [ ]:
class bookingSpider(scrapy.Spider):
    name = "booking"  
   
    cities = cities_france
    
    # get 3 pages by cities (page 0, 25, 60)
    
    start_urls = ["https://www.booking.com/searchresults.html"+ "?nflt=ht_id&" + urlencode({"ss":v,"offset":p}) for v in cities for p in [0,25,60]]
    allowed_domains = ['booking.com']
    
    try:
        def parse(self, response):
           
            print("procesing:"+response.url)
            
            # hotel names
            names = response.css('div.fcab3ed991.a23c043802::text').getall()
            
            # hotel grades
            notes = response.css('div.b5cd09854e.d10a6220b4::text').getall()
           
            # urls link
            all_links =  response.css('a.e13098a59f::attr(href)').getall()
           
                               
                              
            # cities name
            
            cities= response.url
            v = urlparse(cities)
            loc = re.sub(r"&offset=0|&offset=25|&offset=60", "", v.query[14:])
            loc = loc.replace("+"," ") 
            
                
            for name, note, link, in zip(names, notes, all_links) : 
                yield {'city':loc,
                    'hotel_name': name,
                       'score': note,
                      'url':link}
            
        
   
                
            
        
    except:
        pass
    
process.crawl(bookingSpider)
process.start()

hotels = pd.read_json('scrap_booking.json')

procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=Mont+Saint+Michel&offset=25
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=Bayeux&offset=0
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=St+Malo&offset=0
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=Bayeux&offset=25
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=Mont+Saint+Michel&offset=60
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=St+Malo&offset=60
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=Mont+Saint+Michel&offset=0
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=St+Malo&offset=25
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=Bayeux&offset=60
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=Le+Havre&offset=0
procesing:https://www.booking.com/searchresults.html?nflt=ht_id&ss=Le+Havre&offset=25
procesing:https://www.booking.com/search

In [ ]:
hotels

,city,hotel_name,score,url
0,Mont Saint Michel,Maison au pied du Mont Saint Michel,9.1,https://www.booking.com/hotel/fr/maison-au-pie...
1,Mont Saint Michel,Au Mont Pour Deux,8.2,https://www.booking.com/hotel/fr/au-mont-pour-...
2,Mont Saint Michel,Apparthôtel Mont Saint Michel - Résidence Fleu...,8.0,https://www.booking.com/hotel/fr/residence-fle...
3,Mont Saint Michel,Les Pres-Salés,9.0,https://www.booking.com/hotel/fr/les-pra-s-sal...
4,Mont Saint Michel,Gîtes le Mont Desclos Saint Michel,9.3,https://www.booking.com/hotel/fr/gites-le-mont...
...,...,...,...,...
2499,La Rochelle,"Hotel La Marine, Vieux Port",8.2,https://www.booking.com/hotel/fr/la-marine-la-...
2500,La Rochelle,Cit'Hotel le Bord'O Vieux Port,8.2,https://www.booking.com/hotel/fr/hoteldebordea...
2501,La Rochelle,La Résidence des Indes,9.0,https://www.booking.com/hotel/fr/la-residence-...
2502,La Rochelle,Central Park Hôtel & Spa,8.7,https://www.booking.com/hotel/fr/central-park-...


In [ ]:
liste_url = hotels.url.tolist()
liste_url[0]

'https://www.booking.com/hotel/fr/maison-au-pied-du-mont-saint-michel.en-gb.html?aid=304142&label=gen173nr-1FCAQoggJCGHNlYXJjaF9tb250IHNhaW50IG1pY2hlbEgzWARoTYgBAZgBCbgBGcgBD9gBAegBAfgBA4gCAagCA7gC3Nj9mgbAAgHSAiQxZDY1NzY1OS01ZGUxLTQxYjYtODBiZC00M2U2N2MzZGY1ODfYAgXgAgE&ucfs=1&arphpl=1&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=26&sr_order=popularity&nflt=ht_id&srpvid=25b22e2e219f0158&srepoch=1667198045&from=searchresults#hotelTmpl'

In [ ]:
localisations =[]
descript=[]




for url1 in liste_url :
    head = {"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36"}

    reponse1 = requests.get(url1, headers=head)
    soup_bis= BeautifulSoup(reponse1.text,"html.parser")
    
  
        
    try:
        # get latitude and longitude
        localisation = soup_bis.find("a", id = "hotel_address")
        localisations.append([float(l) for l in localisation["data-atlas-latlng"].split(",")])
         
        
    except:
                    
        localisations.append(None)
        
        
    try:
        # hotel description
        description = soup_bis.find_all("div", class_ = "hp_desc_main_content")  
        d=[dc.get_text() for dc in description[0].find_all("p")]
        d =[re.sub(r"\n|\xa0&|\xa0|\xa0-", " ", w) for w in d]
        descript.append(d)
    except:
        
        descript.append(None)

Create latitude and longitude column
Delete localisation


In [ ]:
hotels["localisation"] = localisations
hotels["longitude"] =[l[1] for l  in hotels["localisation"]]
hotels["latitude"] =[l[0] for l in hotels["localisation"]]
hotels.drop("localisation",axis=1, inplace= True)
hotels["description"]=descript


In [ ]:
def replace_none(x):
    if x == None:
        x = []
    return x

hotels["description"]= hotels["description"].apply(lambda x : replace_none(x))

In [ ]:
dico_isnull = {c : (len(hotels[hotels[c].isnull()== True])/(hotels.shape[0])) * 100 for c in hotels.columns}
dico_isnull

{'city': 0.0,
 'hotel_name': 0.0,
 'score': 0.0,
 'url': 0.0,
 'longitude': 0.0,
 'latitude': 0.0,
 'description': 0.0}

In [ ]:
hotels["description"] = hotels["description"].apply(lambda x: ' '.join(map(str, x)))

In [ ]:
hotels = hotels.sort_values(by = ['city'])
t = [hotels[hotels.city == v].sort_values(by = ['score'],ascending = False)for v in cities_france]
t[5]

,city,hotel_name,score,url,longitude,latitude,description
456,Paris,Edgar Suites Louvre - Jour,9.3,https://www.booking.com/hotel/fr/edgar-suites-...,2.345075,48.864076,Situated less than 1 km from Pompidou Centre i...
416,Paris,La Maison de Léa,9.3,https://www.booking.com/hotel/fr/la-maison-de-...,2.374252,48.856711,You're eligible for a Genius discount at La Ma...
448,Paris,My Maison In Paris - Louvre,9.2,https://www.booking.com/hotel/fr/my-maison-in-...,2.343194,48.863615,You're eligible for a Genius discount at My Ma...
418,Paris,LUXE EIFFEL TOWER AND SEINE RIVER,9.2,https://www.booking.com/hotel/fr/luxe-eiffel-t...,2.271436,48.844192,You're eligible for a Genius discount at LUXE ...
381,Paris,Appartement Petits Champs,9.1,https://www.booking.com/hotel/fr/appartement-p...,2.340397,48.864306,"Situated in the centre of Paris, Appartement P..."
...,...,...,...,...,...,...,...
454,Paris,LivinParis - Luxury 4 Bedrooms Le Marais I,7.4,https://www.booking.com/hotel/fr/luxury-4-bedr...,2.355439,48.868388,"Situated in the centre of Paris, within 1.2 km..."
438,Paris,Les Patios du Marais 1,7.3,https://www.booking.com/hotel/fr/les-patios-du...,2.356815,48.864264,You're eligible for a Genius discount at Les P...
415,Paris,Hotel Le Clos d'Alésia,7.0,https://www.booking.com/hotel/fr/le-clos-d-ala...,2.324461,48.826125,Ideally situated in the 14th arr. District of ...
452,Paris,Hotel Victoria,7.0,https://www.booking.com/hotel/fr/victoria-pari...,2.343294,48.871930,You're eligible for a Genius discount at Hotel...


Merge dataframes of each hotel to one finale dataframe

In [ ]:
info_hotels_by_city  = pd.concat(t)

info_hotels_by_city 

,city,hotel_name,score,url,longitude,latitude,description
58,Mont Saint Michel,Maison proche Mont Saint Michel,10.0,https://www.booking.com/hotel/fr/maison-proche...,-1.504222,48.597071,"Maison proche Mont Saint Michel, a property wi..."
73,Mont Saint Michel,L'Aurore de la Baie,9.7,https://www.booking.com/hotel/fr/l-39-aurore-d...,-1.452512,48.609807,"Providing garden views, L'Aurore de la Baie in..."
59,Mont Saint Michel,Gîtes le Mont Desclos Saint Michel,9.6,https://www.booking.com/hotel/fr/gites-le-mont...,-1.504449,48.596656,"Situated in Beauvoir, 5.7 km from Mont Saint M..."
65,Mont Saint Michel,Chambres Les Salles,9.6,https://www.booking.com/hotel/fr/chambres-les-...,-1.501436,48.595026,You're eligible for a Genius discount at Chamb...
60,Mont Saint Michel,"Maison chaleureuse, spacieuse et familiale",9.6,https://www.booking.com/hotel/fr/maison-chaleu...,-1.505204,48.596577,"Maison chaleureuse, spacieuse et familiale in ..."
...,...,...,...,...,...,...,...
2440,La Rochelle,Appart'City La Rochelle,7.2,https://www.booking.com/hotel/fr/appart-city-c...,-1.141012,46.158447,This Appart'city is a 10-minute walk from La R...
2457,La Rochelle,Résidence Pierre & Vacances Centre,7.2,https://www.booking.com/hotel/fr/pierre-vacanc...,-1.146392,46.155769,"Located in La Rochelle, this Pierre & Vacances..."
2461,La Rochelle,Lagrange Apart'Hotel l’Escale Marine,7.2,https://www.booking.com/hotel/fr/l-escale-mari...,-1.154863,46.151070,You're eligible for a Genius discount at Lagra...
2453,La Rochelle,Hôtel Le Rupella,7.0,https://www.booking.com/hotel/fr/le-rupella.en...,-1.151070,46.158175,"Located in the heart of La Rochelle, just a 10..."


Ad ID City to dataframe

In [ ]:
def id_city (x): 
    if x == "Mont Saint Michel":
        x = 1
    if x == "St Malo":
        x = 2
    if x == "Bayeux":
        x = 3
    if x == "Le Havre":
        x=4
    if x== "Rouen":
        x=5
    if x== "Paris":
        x=6
    if x== "Amiens":
        x=7
    if x=="Lille":
        x=8
    if x =="Strasbourg":
        x=9
    if x=="Chateau du Haut Koenigsbourg":
        x=10
    if x== "Colmar":
        x=11
    if x=="Eguisheim":
        x=12
    if x=="Besancon":
        x=13
    if x=="Dijon":
        x=14
    if x=="Annecy":
        x=15
    if x=="Grenoble":
        x=16
    if x=="Lyon":
        x=17
    if x=="Gorges du Verdon":
        x=18
    if x=="Bormes les Mimosas":
        x=19
    if x=="Cassis":
        x=20
    if x=="Marseille":
        x=21
    if x=="Aix en Provence":
        x=22
    if x=="Avignon":
        x=23
    if x=="Uzes":
        x=24
    if x=="Nimes":
        x=25
    if x=="Aigues Mortes":
        x=26
    if x== "Saintes Maries de la mer":
        x=27
    if x== "Collioure":
        x=28
    if x=="Carcassonne":
        x=29
    if x=="Ariege":
        x=30
    if x=="Toulouse":
        x=31
    if x=="Montauban":
        x=32
    if x=="Biarritz":
        x=33
    if x=="Bayonne":
        x=34
    if x=="La Rochelle":
        x=35
    
    return x

In [ ]:
info_hotels_by_city ["id_city"] = info_hotels_by_city ["city"].apply(lambda x : id_city(x))


Final table 

In [ ]:
info_hotels_by_city

,city,hotel_name,score,url,longitude,latitude,description,id_city
58,Mont Saint Michel,Maison proche Mont Saint Michel,10.0,https://www.booking.com/hotel/fr/maison-proche...,-1.504222,48.597071,"Maison proche Mont Saint Michel, a property wi...",1
73,Mont Saint Michel,L'Aurore de la Baie,9.7,https://www.booking.com/hotel/fr/l-39-aurore-d...,-1.452512,48.609807,"Providing garden views, L'Aurore de la Baie in...",1
59,Mont Saint Michel,Gîtes le Mont Desclos Saint Michel,9.6,https://www.booking.com/hotel/fr/gites-le-mont...,-1.504449,48.596656,"Situated in Beauvoir, 5.7 km from Mont Saint M...",1
65,Mont Saint Michel,Chambres Les Salles,9.6,https://www.booking.com/hotel/fr/chambres-les-...,-1.501436,48.595026,You're eligible for a Genius discount at Chamb...,1
60,Mont Saint Michel,"Maison chaleureuse, spacieuse et familiale",9.6,https://www.booking.com/hotel/fr/maison-chaleu...,-1.505204,48.596577,"Maison chaleureuse, spacieuse et familiale in ...",1
...,...,...,...,...,...,...,...,...
2440,La Rochelle,Appart'City La Rochelle,7.2,https://www.booking.com/hotel/fr/appart-city-c...,-1.141012,46.158447,This Appart'city is a 10-minute walk from La R...,35
2457,La Rochelle,Résidence Pierre & Vacances Centre,7.2,https://www.booking.com/hotel/fr/pierre-vacanc...,-1.146392,46.155769,"Located in La Rochelle, this Pierre & Vacances...",35
2461,La Rochelle,Lagrange Apart'Hotel l’Escale Marine,7.2,https://www.booking.com/hotel/fr/l-escale-mari...,-1.154863,46.151070,You're eligible for a Genius discount at Lagra...,35
2453,La Rochelle,Hôtel Le Rupella,7.0,https://www.booking.com/hotel/fr/le-rupella.en...,-1.151070,46.158175,"Located in the heart of La Rochelle, just a 10...",35


Save to csv

In [ ]:
info_hotels_by_city.to_csv('info_hotels_by_city.csv')

In [ ]:
info_hotels_by_city = pd.read_csv("info_hotels_by_city.csv")

In [ ]:
hotels_trip_final =info_hotels_by_city.merge(data_weather_cities_class, on=["id_city","city"], how = "outer")
hotels_trip_final

Get 20 best hotels from the best 35 cities

In [23]:
liste_hotels_all= [hotels_trip_final[hotels_trip_final.city == town].iloc[0:20] for town in cities_france]


best_hotels_all = pd.concat(liste_hotels_all)
best_hotels_all 

,city,hotel_name,score,url,longitude_x,latitude_x,description,id_city,latitude_y,longitude_y,Average probability of precipitation,average cloud percentage,average daily temperatures
0,Mont Saint Michel,Maison proche Mont Saint Michel,10.0,https://www.booking.com/hotel/fr/maison-proche...,-1.504222,48.597071,"Maison proche Mont Saint Michel, a property wi...",1,48.635954,-1.511460,0.828750,70.875000,18.335000
1,Mont Saint Michel,L'Aurore de la Baie,9.7,https://www.booking.com/hotel/fr/l-39-aurore-d...,-1.452512,48.609807,"Providing garden views, L'Aurore de la Baie in...",1,48.635954,-1.511460,0.828750,70.875000,18.335000
2,Mont Saint Michel,Gîtes le Mont Desclos Saint Michel,9.6,https://www.booking.com/hotel/fr/gites-le-mont...,-1.504449,48.596656,"Situated in Beauvoir, 5.7 km from Mont Saint M...",1,48.635954,-1.511460,0.828750,70.875000,18.335000
3,Mont Saint Michel,Chambres Les Salles,9.6,https://www.booking.com/hotel/fr/chambres-les-...,-1.501436,48.595026,You're eligible for a Genius discount at Chamb...,1,48.635954,-1.511460,0.828750,70.875000,18.335000
4,Mont Saint Michel,"Maison chaleureuse, spacieuse et familiale",9.6,https://www.booking.com/hotel/fr/maison-chaleu...,-1.505204,48.596577,"Maison chaleureuse, spacieuse et familiale in ...",1,48.635954,-1.511460,0.828750,70.875000,18.335000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2450,La Rochelle,Vue Superbe sur Tour de la Lanterne T2 + Terrasse,9.0,https://www.booking.com/hotel/fr/vue-superbe-s...,-1.158386,46.155847,"Situated in La Rochelle, just 200 m from Concu...",35,46.159113,-1.152043,0.530786,67.796429,20.123643
2451,La Rochelle,Villa Verde La Rochelle,9.0,https://www.booking.com/hotel/fr/villa-verde-l...,-1.160173,46.162524,"Boasting a terrace, Villa Verde La Rochelle is...",35,46.159113,-1.152043,0.530786,67.796429,20.123643
2452,La Rochelle,Petite Roche,8.9,https://www.booking.com/hotel/fr/petite-roche....,-1.146452,46.159879,You're eligible for a Genius discount at Petit...,35,46.159113,-1.152043,0.530786,67.796429,20.123643
2453,La Rochelle,Appartement avec cour privée Centre ville Clas...,8.8,https://www.booking.com/hotel/fr/appartement-a...,-1.158386,46.155847,"Set in La Rochelle, 200 m from Concurrence, 1....",35,46.159113,-1.152043,0.530786,67.796429,20.123643


Maps of the best 20 hotels for each citites

In [ ]:
fig = px.scatter_mapbox(best_hotels_all, lat="latitude_x", lon="longitude_x", color="city",
                        mapbox_style="open-street-map",                    
                        zoom=10,
                        height=600)
fig.update_layout(
    
    margin=dict(l=30, r=30, t=30, b=30),
    paper_bgcolor="LightSteelBlue",
)
fig.show()
fig.write_html("top20hotelsbycity.html")

Create csv file from our final table

In [13]:
hotels_trip_final.to_csv('hotels_trip_final.csv',index = False) 

In [12]:
hotels_trip_final = pd.read_csv("hotels_trip_final.csv")
hotels_trip_final

,city,hotel_name,score,url,longitude_x,latitude_x,description,id_city,latitude_y,longitude_y,Average probability of precipitation,average cloud percentage,average daily temperatures
0,Mont Saint Michel,Maison proche Mont Saint Michel,10.0,https://www.booking.com/hotel/fr/maison-proche...,-1.504222,48.597071,"Maison proche Mont Saint Michel, a property wi...",1,48.635954,-1.511460,0.828750,70.875000,18.335000
1,Mont Saint Michel,L'Aurore de la Baie,9.7,https://www.booking.com/hotel/fr/l-39-aurore-d...,-1.452512,48.609807,"Providing garden views, L'Aurore de la Baie in...",1,48.635954,-1.511460,0.828750,70.875000,18.335000
2,Mont Saint Michel,Gîtes le Mont Desclos Saint Michel,9.6,https://www.booking.com/hotel/fr/gites-le-mont...,-1.504449,48.596656,"Situated in Beauvoir, 5.7 km from Mont Saint M...",1,48.635954,-1.511460,0.828750,70.875000,18.335000
3,Mont Saint Michel,Chambres Les Salles,9.6,https://www.booking.com/hotel/fr/chambres-les-...,-1.501436,48.595026,You're eligible for a Genius discount at Chamb...,1,48.635954,-1.511460,0.828750,70.875000,18.335000
4,Mont Saint Michel,"Maison chaleureuse, spacieuse et familiale",9.6,https://www.booking.com/hotel/fr/maison-chaleu...,-1.505204,48.596577,"Maison chaleureuse, spacieuse et familiale in ...",1,48.635954,-1.511460,0.828750,70.875000,18.335000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2503,La Rochelle,Appart'City La Rochelle,7.2,https://www.booking.com/hotel/fr/appart-city-c...,-1.141012,46.158447,This Appart'city is a 10-minute walk from La R...,35,46.159113,-1.152043,0.530786,67.796429,20.123643
2504,La Rochelle,Résidence Pierre & Vacances Centre,7.2,https://www.booking.com/hotel/fr/pierre-vacanc...,-1.146392,46.155769,"Located in La Rochelle, this Pierre & Vacances...",35,46.159113,-1.152043,0.530786,67.796429,20.123643
2505,La Rochelle,Lagrange Apart'Hotel l’Escale Marine,7.2,https://www.booking.com/hotel/fr/l-escale-mari...,-1.154863,46.151070,You're eligible for a Genius discount at Lagra...,35,46.159113,-1.152043,0.530786,67.796429,20.123643
2506,La Rochelle,Hôtel Le Rupella,7.0,https://www.booking.com/hotel/fr/le-rupella.en...,-1.151070,46.158175,"Located in the heart of La Rochelle, just a 10...",35,46.159113,-1.152043,0.530786,67.796429,20.123643


In [ ]:
pip install boto3


Bucket AWS

In [ ]:
import boto3

# Create bucket AWS

session = boto3.Session(aws_access_key_id="", aws_secret_access_key="")
s3 = session.resource("s3")
s3.create_bucket(Bucket="")

s3.Bucket(name='bucketkayakrousteau')

Send CSV to S3

In [ ]:
## Add csv file in S3

data = open('hotels_trip_final.csv', 'rb')
s3.Bucket('').put_object(Key='hotels_trip_final.csv', Body=data) # stock le fichier csv dans le bucket dans le service S3

s3.Object(bucket_name='bucketkayakrousteau', key='hotels_trip_final.csv')

In [ ]:
!pip install psycopg2-binary


     ---------------------------------------- 1.2/1.2 MB 3.5 MB/s eta 0:00:00


Create engine and send csv file to SQL database

In [ ]:
from sqlalchemy import create_engine  

hostname = '' 
password = '' 
username = '' 
dbname = ''  # Replace YOUR_USERNAME, YOUR_PASSWORD and YOUR_HOSTNAME # by your credentials 

engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{hostname}/{dbname}", echo=True)

In [ ]:
# Send csv file in SQL database
hotels_trip_final.to_sql('hotels_by_cities', con=engine)